<a href="https://colab.research.google.com/github/miczkejedrzej/MNLP-project-1/blob/main/Cleaning_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import every dataset

In [35]:
from google.colab import files
uploaded = files.upload()

import pandas as pd
from PIL import Image
from torchvision import transforms
import numpy as np

Saving [MNLP 2025 HW1] train set [PUBLIC] - train_cleaned.tsv to [MNLP 2025 HW1] train set [PUBLIC] - train_cleaned.tsv


# Training set

In [36]:
train_df = pd.read_csv('[MNLP 2025 HW1] train set [PUBLIC] - train_cleaned.tsv', sep='\t')

train_df_subclass = pd.read_json('trainset_subclass_instance.json', orient="records", lines=True)
train_df_dates = pd.read_json('train_df_dates.json', orient="records", lines=True)
train_df_lang = pd.read_json('train_df_lang.json', orient="records", lines=True)
train_df_country = pd.read_json('train_df_country.json', orient="records", lines=True)
train_df_descr = pd.read_json('train_df_descr_analyse.json', orient="records", lines=True)

## Merge all the datasets

In [37]:
train_df.columns.to_list()

['item', 'name', 'description', 'type', 'category', 'subcategory', 'label']

In [38]:
# Get the dataset we first had
initial_df = train_df
initial_variables = train_df.columns.to_list()

# Get the datasets with the features we created
df_to_merge = [train_df_subclass, train_df_dates, train_df_lang, train_df_country, train_df_descr]

# Merge all of them, in a single dataframe
for df in df_to_merge:
  initial_df = pd.merge(left=initial_df, right=df, on=initial_variables, how='inner')

# Visualisation :
initial_df.head()

,item,name,description,type,category,subcategory,label,subclasses,instances_of,subclass_depth,image,date,nb_lang,main_country,descr_num_nouns,descr_num_verbs,descr_num_adjectives,descr_has_location,descr_has_ethnic_group,descr_has_event
0,http://www.wikidata.org/entity/Q306,Sebastián Piñera,Chilean entrepreneur and politician (1949–2024),entity,politics,politician,cultural exclusive,0,1,1,"[[[89, 90, 72], [95, 98, 79], [76, 81, 60], [6...",1949.0,126.0,Q298,2,0,1,0,1,0
1,http://www.wikidata.org/entity/Q12735,John Amos Comenius,"Czech teacher, educator, philosopher and write...",entity,politics,politician,cultural representative,0,1,1,"[[[33, 31, 16], [34, 32, 17], [37, 35, 19], [3...",1592.0,72.0,Q153136,4,0,1,0,1,0
2,http://www.wikidata.org/entity/Q1752,Macrinus,Roman emperor from 217 to 218,entity,politics,politician,cultural representative,0,1,1,"[[[111, 112, 104], [111, 112, 104], [113, 114,...",165.0,83.0,Q1747689,1,0,1,0,1,0
3,http://www.wikidata.org/entity/Q1639,Lamine Diack,Senegalese sports manager (1933–2021),entity,politics,politician,cultural representative,0,1,1,"[[[126, 82, 35], [128, 82, 33], [130, 83, 32],...",1933.0,42.0,Q1041,2,0,1,0,1,0
4,http://www.wikidata.org/entity/Q9588,Richard Nixon,President of the United States from 1969 to 1974,entity,politics,politician,cultural representative,0,1,1,"[[[106, 88, 76], [106, 88, 76], [108, 90, 78],...",1913.0,174.0,Q30,0,0,0,1,0,0


## Cleaning

### Type

In [39]:
var = 'type'
initial_df[var].describe()

,type
count,6251
unique,2
top,entity
freq,3924


In [40]:
initial_df[var].unique()

array(['entity', 'concept'], dtype=object)

In [41]:
initial_df[var].isna().sum()

np.int64(0)

We convert the variable 'type' to binary so that we can use it in our models.

In [42]:
initial_df['type_cat'] = initial_df['type'].apply(lambda x: 1 if x == 'entity' else 0 if x == 'concept' else None)

### Category

In [43]:
var = 'category'
initial_df[var].describe()

,category
count,6251
unique,19
top,politics
freq,532


In [44]:
initial_df[var].unique()

array(['politics', 'food', 'architecture', 'transportation', 'sports',
       'media', 'philosophy and religion', 'literature', 'visual arts',
       'fashion', 'comics and anime', 'performing arts', 'music', 'films',
       'books', 'history', 'gestures and habits', 'biology', 'geography'],
      dtype=object)

In [45]:
initial_df[var].isna().sum()

np.int64(0)

We map the variable 'category' to multimodal variable, so that we can use it in our models.

In [46]:
# Get the possible categories
list_var = initial_df[var].unique().tolist()

# Mapping
mapping_category = {val: idx for idx, val in enumerate(list_var)}

# Print the map, to keep it in mind
for val, idx in mapping_category.items():
    print(f"{val} : {idx}")

initial_df["category_cat"] = initial_df['category'].map(mapping_category)

politics : 0
food : 1
architecture : 2
transportation : 3
sports : 4
media : 5
philosophy and religion : 6
literature : 7
visual arts : 8
fashion : 9
comics and anime : 10
performing arts : 11
music : 12
films : 13
books : 14
history : 15
gestures and habits : 16
biology : 17
geography : 18


### Subcategory

In [47]:
var = 'subcategory'
initial_df[var].describe()

,subcategory
count,6251
unique,108
top,politician
freq,99


In [48]:
initial_df[var].unique()

array(['politician', 'political party', 'government agency', 'law',
       'policy', 'government', 'food', 'drink', 'dish', 'ingredient',
       'cook', 'cooking technique', 'architectural structure',
       'architectural style', 'architect', 'construction', 'building',
       'building material', 'transport company',
       'automobile manufacturer', 'transport', 'mode of transport',
       'station', 'athlete', 'sports team', 'sport', 'sports club',
       'recurring sporting event', 'sports equipment', 'media company',
       'magazine', 'television', 'production company', 'film studio',
       'streaming service', 'philosopher', 'religious leader', 'religion',
       'philosophy', 'philosophical movement', 'religious movement',
       'poetry', 'poet', 'writing style', 'literary genre',
       'literary award', 'writer', 'publisher', 'artist', 'visual arts',
       'art gallery', 'art movement', 'painting', 'photographer',
       'clothing', 'traditional costume', 'designer', 'fas

In [49]:
initial_df[var].isna().sum()

np.int64(0)

We map the variable 'category' to multimodal variable, so that we can use it in our models.

In [50]:
# Get the possible categories
list_var = initial_df[var].unique().tolist()

# Mapping
mapping_subcategory = {val: idx for idx, val in enumerate(list_var)}

# Print the map, to keep it in mind
for val, idx in mapping_subcategory.items():
    print(f"{val} : {idx}")

initial_df["subcategory_cat"] = initial_df['subcategory'].map(mapping_subcategory)

politician : 0
political party : 1
government agency : 2
law : 3
policy : 4
government : 5
food : 6
drink : 7
dish : 8
ingredient : 9
cook : 10
cooking technique : 11
architectural structure : 12
architectural style : 13
architect : 14
construction : 15
building : 16
building material : 17
transport company : 18
automobile manufacturer : 19
transport : 20
mode of transport : 21
station : 22
athlete : 23
sports team : 24
sport : 25
sports club : 26
recurring sporting event : 27
sports equipment : 28
media company : 29
magazine : 30
television : 31
production company : 32
film studio : 33
streaming service : 34
philosopher : 35
religious leader : 36
religion : 37
philosophy : 38
philosophical movement : 39
religious movement : 40
poetry : 41
poet : 42
writing style : 43
literary genre : 44
literary award : 45
writer : 46
publisher : 47
artist : 48
visual arts : 49
art gallery : 50
art movement : 51
painting : 52
photographer : 53
clothing : 54
traditional costume : 55
designer : 56
fashi

### Labels

In [52]:
initial_df['label'].describe()

,label
count,6244
unique,9
top,cultural exclusive
freq,2685


In [53]:
initial_df['label'].unique()

array(['cultural exclusive', 'cultural representative',
       'cultural agnostic', 'cultural', 'cult', nan, 'cultural agn',
       'cultural represent', 'cultural ex', 'cultural ag'], dtype=object)

In [54]:
initial_df[['item', 'label']].groupby("label").count()

,item
label,
cult,1
cultural,5
cultural ag,1
cultural agn,3
cultural agnostic,1862
cultural ex,1
cultural exclusive,2685
cultural represent,1
cultural representative,1685


We are suppose to have only 3 different labels

- Cultural Exclusive (C.E.)
- Cultural Agnostic (C.A.)
- Cultural Representative (C.R.)

Instead, we have seen that there are 9 different labels in the dataset, plus some NaN. So we have to clean the data, in order to get only 3 differents labels. Here is the solution we propose :

First step :

We distinguish the 3 labels we want (C.A., C.E., C.R.) and also labels that are close to this ('Cultural agn', 'cultural represent', 'cultural ex', 'cultural ag'). So we propose the following mapping :

- 'cultural agn' : C.A.
- 'cultural represent' : C.R.
- 'cultural ex' : C.E.
- 'cultural ag' : C.A.

Second step :

For the other labels ('cultural', 'cult', nan), we propose to apply the rule that was define to build the dataset (see https://huggingface.co/datasets/sapienzanlp/nlp2025_hw1_cultural_dataset) : Ask ChatGPT-o3 !

First step :

In [55]:
to_correct = initial_df[initial_df["label"].isin(['cultural agn','cultural represent', 'cultural ex', 'cultural ag'])][['item', 'label']]
to_correct

,item,label
1545,http://www.wikidata.org/entity/Q86135347,cultural agn
3698,http://www.wikidata.org/entity/Q509900,cultural represent
4391,http://www.wikidata.org/entity/Q643677,cultural ex
4819,http://www.wikidata.org/entity/Q1711593,cultural agn
5731,http://www.wikidata.org/entity/Q25618,cultural ag
5968,http://www.wikidata.org/entity/Q30405,cultural agn


In [56]:
mapping_correction = {
  'cultural agn' : 'cultural agnostic',
  'cultural represent' : 'cultural representative',
  'cultural ex' : 'cultural exclusive',
  'cultural ag' : 'cultural agnostic'
}

In [57]:
to_correct['label'] = to_correct['label'].map(mapping_correction)

In [58]:
# We update the transformations to the initial dataset
initial_df.set_index('item', inplace=True)
to_correct.set_index('item', inplace=True)

initial_df.update(to_correct)
initial_df.reset_index(inplace=True)


# We check if the updates are corrects
initial_df[1543:1547]

,item,name,description,type,category,subcategory,label,subclasses,instances_of,subclass_depth,...,main_country,descr_num_nouns,descr_num_verbs,descr_num_adjectives,descr_has_location,descr_has_ethnic_group,descr_has_event,type_cat,category_cat,subcategory_cat
1543,http://www.wikidata.org/entity/Q27503001,professional athlete,person who earns their living from sports,concept,sports,athlete,cultural agnostic,0,1,9,...,None,3,1,0,0,0,0,0,4,23
1544,http://www.wikidata.org/entity/Q107690317,competition climber,climber who competes in IFSC and Olympic climb...,concept,sports,athlete,cultural agnostic,0,3,9,...,None,2,2,1,0,0,1,0,4,23
1545,http://www.wikidata.org/entity/Q86135347,climber,person who practices climbing,concept,sports,athlete,cultural agnostic,2,1,9,...,None,1,2,0,0,0,0,0,4,23
1546,http://www.wikidata.org/entity/Q28971125,cheerleader,performer who leads crowd support at sports ev...,concept,sports,athlete,cultural representative,0,1,9,...,None,9,2,0,0,0,0,0,4,23


Second step :

Here are the items that ChatGPT will have to classify :

In [59]:
to_classify = initial_df[(initial_df["label"].isin(["cultural", "cult"])) | (initial_df["label"].isna())][['item', 'name', 'description']]

to_classify

,item,name,description
858,http://www.wikidata.org/entity/Q957033,Sunny Baudelaire,fictional character
1066,http://www.wikidata.org/entity/Q30327019,post and lintel,building system where horizontal elements (bea...
1214,http://www.wikidata.org/entity/Q811361,architectural glass,building material typically used as transparen...
1651,http://www.wikidata.org/entity/Q12014207,trekking,backpacking or hiking
2556,http://www.wikidata.org/entity/Q257907,Ethiopian movement,religious movement in southern Africa
2831,http://www.wikidata.org/entity/Q1136336,Costa Book Awards,annual series of literary awards in five categ...
3236,http://www.wikidata.org/entity/Q67111,Franz Pfemfert,German journalist (1879-1954)
3820,http://www.wikidata.org/entity/Q1089672,The Elm-Chanted Forest,1986 animated film directed by Milan Blažeković
4853,http://www.wikidata.org/entity/Q85755629,Daniel Airlie,novel
5566,http://www.wikidata.org/entity/Q206912,extremophile,organisms specifically adapted to live and sur...


Here is the result for chatgpt :

In [60]:
categories = {
    'http://www.wikidata.org/entity/Q957033': 'cultural representative',
    'http://www.wikidata.org/entity/Q30327019': 'cultural agnostic',
    'http://www.wikidata.org/entity/Q811361': 'cultural agnostic',
    'http://www.wikidata.org/entity/Q12014207': 'cultural agnostic',
    'http://www.wikidata.org/entity/Q257907': 'cultural exclusive',
    'http://www.wikidata.org/entity/Q1136336': 'cultural representative',
    'http://www.wikidata.org/entity/Q67111': 'cultural representative',
    'http://www.wikidata.org/entity/Q1089672': 'cultural representative',
    'http://www.wikidata.org/entity/Q85755629': 'cultural representative',
    'http://www.wikidata.org/entity/Q206912': 'cultural agnostic',
    'http://www.wikidata.org/entity/Q23228': 'cultural agnostic',
    'http://www.wikidata.org/entity/Q3196604': 'cultural representative',
    'http://www.wikidata.org/entity/Q1940624': 'cultural representative'
}

to_classify['label'] = to_classify['item'].map(categories)

In [61]:
# We update the transformations to the initial dataset
initial_df.set_index('item', inplace=True)
to_classify.set_index('item', inplace=True)

initial_df.update(to_classify)
initial_df.reset_index(inplace=True)


# We check if the updates are corrects
initial_df[856:860]

,item,name,description,type,category,subcategory,label,subclasses,instances_of,subclass_depth,...,main_country,descr_num_nouns,descr_num_verbs,descr_num_adjectives,descr_has_location,descr_has_ethnic_group,descr_has_event,type_cat,category_cat,subcategory_cat
856,http://www.wikidata.org/entity/Q929866,Santi Santamaria,Spanish chef (1957–2011),entity,food,cook,cultural exclusive,0,1,1,...,Q29,1,0,1,0,1,0,1,1,10
857,http://www.wikidata.org/entity/Q899011,Ian Beale,fictional character from the soap opera EastEn...,entity,food,cook,cultural exclusive,0,2,1,...,None,3,0,1,0,0,0,1,1,10
858,http://www.wikidata.org/entity/Q957033,Sunny Baudelaire,fictional character,entity,food,cook,cultural representative,0,4,1,...,None,1,0,1,0,0,0,1,1,10
859,http://www.wikidata.org/entity/Q935079,SpongeBob SquarePants,main character from the animated television sh...,entity,food,cook,cultural representative,0,4,1,...,None,3,1,1,0,0,0,1,1,10


Check if everything is normal now :

In [62]:
initial_df['label'].describe()

,label
count,6251
unique,3
top,cultural exclusive
freq,2687


In [63]:
initial_df['label'].unique()

array(['cultural exclusive', 'cultural representative',
       'cultural agnostic'], dtype=object)

In [64]:
initial_df[['item', 'label']].groupby("label").count()

,item
label,
cultural agnostic,1871
cultural exclusive,2687
cultural representative,1693


### Images

Here, we have a lot of missing values, so we fill in the missing values with white images, in order to

- Hope that the model will learn to recognize and ignore thoses white images
- Interpret the fact that the absence of image for an item can be correlated to its label
- Regularization with noise

In [65]:
#def fill_NaN_image(image):

#  """
#  If a cell is empty, it fills in with a white image, of dimension (28,28,3) ie (height, width, channels)
#  """

#  if image is None or (isinstance(image, float) and np.isnan(image)):
#    image = np.ones((128, 128, 3), dtype=np.uint8) * 255

#  return image

In [66]:
#initial_df['image'] = initial_df['image'].apply(lambda x: fill_NaN_image(x))

In [68]:
#initial_df[['item', 'image']]

,item,image
0,http://www.wikidata.org/entity/Q306,"[[[89, 90, 72], [95, 98, 79], [76, 81, 60], [6..."
1,http://www.wikidata.org/entity/Q12735,"[[[33, 31, 16], [34, 32, 17], [37, 35, 19], [3..."
2,http://www.wikidata.org/entity/Q1752,"[[[111, 112, 104], [111, 112, 104], [113, 114,..."
3,http://www.wikidata.org/entity/Q1639,"[[[126, 82, 35], [128, 82, 33], [130, 83, 32],..."
4,http://www.wikidata.org/entity/Q9588,"[[[106, 88, 76], [106, 88, 76], [108, 90, 78],..."
...,...,...
6246,http://www.wikidata.org/entity/Q321103,"[[[255, 255, 255], [255, 255, 255], [255, 255,..."
6247,http://www.wikidata.org/entity/Q338167,"[[[255, 255, 255], [255, 255, 255], [255, 255,..."
6248,http://www.wikidata.org/entity/Q66991,"[[[58, 115, 222], [59, 116, 223], [60, 117, 22..."
6249,http://www.wikidata.org/entity/Q66922,"[[[52, 52, 24], [73, 71, 26], [111, 106, 31], ..."


### Dates

In [ ]:
var = 'date'
initial_df[var].describe(include='category')

,date
count,2506.000000
mean,1880.424980
std,258.087493
min,1.000000
25%,1892.000000
50%,1948.000000
75%,1983.000000
max,5000.000000


In [ ]:
initial_df[var].isna().sum()

np.int64(3745)

In [ ]:
type(initial_df[var][0])

numpy.float64

In [ ]:
initial_df[var].nunique()

446

Problems :

- Maximum values = 5000. Not possible : Set it to Nan
- Lot of missing values : Same arguments as for the photos, we fill in with the date -1 : Hope that the model will learn to recognize item with no dates. Interpret the fact that the absence of dates for an item can be correlated to its label


Max values

In [ ]:
current_year = int(2025)
initial_df['date'] = initial_df['date'].apply(lambda x: None if x > current_year else x)

In [ ]:
var = 'date'
initial_df[var].describe()

,date
count,2504.000000
mean,1878.931709
std,250.237383
min,1.000000
25%,1892.000000
50%,1948.000000
75%,1983.000000
max,2025.000000


Missing values

In [ ]:
initial_df['date'] = initial_df['date'].fillna(-1)

### Nb_lang

In [ ]:
var = 'nb_lang'
initial_df[var].describe(include='category')

,nb_lang
count,6250.000000
mean,31.190240
std,36.651012
min,1.000000
25%,8.000000
50%,19.000000
75%,39.000000
max,310.000000


In [ ]:
initial_df[var].isna().sum()

np.int64(1)

In [ ]:
type(initial_df[var][0])

numpy.float64

In [ ]:
initial_df[var].nunique()

219

Problem : We have 1 missing value

In [ ]:
initial_df[initial_df[var].isna()]

,item,name,description,type,category,subcategory,label,subclasses,instances_of,subclass_depth,image,date,nb_lang,main_country,descr_num_nouns,descr_num_verbs,descr_num_adjectives,descr_has_location,descr_has_ethnic_group,descr_has_event
2058,http://www.wikidata.org/entity/Q7551241,social media and television,Emerging platforms,entity,media,television,cultural agnostic,0,2,1,"[[[255, 255, 255], [255, 255, 255], [255, 255,...",-1.0,NaN,None,1,0,1,0,0,0


We fill in with the median

In [ ]:
med_value = initial_df['nb_lang'].median()
initial_df['nb_lang'] = initial_df['nb_lang'].fillna(int(med_value))

### Main country

In [ ]:
var = 'main_country'
initial_df[var].describe()

,main_country
count,3384
unique,283
top,Q30
freq,580


In [ ]:
initial_df[var].isna().sum()

np.int64(2867)

In [ ]:
type(initial_df[var][0])

str

In [ ]:
initial_df[var].nunique()

283

Problem : Many missing values

- Solution : Categorize the missing values in a categorie, and then, for each unique country, we associate a category. Problem : we are not sur that we have collected all the country in the train data set. So, during inference, it would be a problem is the model does not recognize the category

- Solution (what we will do) : 0 if NaN, 1 otherwise. We loose a lot of information...

In [ ]:
initial_df['main_country_cat'] = initial_df['main_country'].apply(lambda x: 0 if pd.isna(x) else 1)

## Exportation

In [ ]:
# Visualisation

initial_df

,item,name,description,type,category,subcategory,label,subclasses,instances_of,subclass_depth,...,date,nb_lang,main_country,descr_num_nouns,descr_num_verbs,descr_num_adjectives,descr_has_location,descr_has_ethnic_group,descr_has_event,main_country_cat
0,http://www.wikidata.org/entity/Q306,Sebastián Piñera,Chilean entrepreneur and politician (1949–2024),entity,politics,politician,cultural exclusive,0,1,1,...,1949.0,126.0,Q298,2,0,1,0,1,0,1
1,http://www.wikidata.org/entity/Q12735,John Amos Comenius,"Czech teacher, educator, philosopher and write...",entity,politics,politician,cultural representative,0,1,1,...,1592.0,72.0,Q153136,4,0,1,0,1,0,1
2,http://www.wikidata.org/entity/Q1752,Macrinus,Roman emperor from 217 to 218,entity,politics,politician,cultural representative,0,1,1,...,165.0,83.0,Q1747689,1,0,1,0,1,0,1
3,http://www.wikidata.org/entity/Q1639,Lamine Diack,Senegalese sports manager (1933–2021),entity,politics,politician,cultural representative,0,1,1,...,1933.0,42.0,Q1041,2,0,1,0,1,0,1
4,http://www.wikidata.org/entity/Q9588,Richard Nixon,President of the United States from 1969 to 1974,entity,politics,politician,cultural representative,0,1,1,...,1913.0,174.0,Q30,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6246,http://www.wikidata.org/entity/Q321103,Bühl,"quarter of Tübingen, Baden-Württemberg, Germany",entity,geography,neighborhood,cultural exclusive,0,2,1,...,-1.0,73.0,Q183,1,0,0,1,0,0,1
6247,http://www.wikidata.org/entity/Q338167,Tenderloin,area of New York City during the late 19th and...,entity,geography,neighborhood,cultural exclusive,0,1,1,...,-1.0,9.0,Q30,3,0,3,1,0,0,1
6248,http://www.wikidata.org/entity/Q66991,Schinznach-Dorf,former municipality and current district of Sc...,entity,geography,neighborhood,cultural exclusive,0,3,1,...,-1.0,30.0,Q39,2,0,2,1,0,0,1
6249,http://www.wikidata.org/entity/Q66922,Ependes,village and former municipality in Bois-d'Amon...,entity,geography,neighborhood,cultural exclusive,0,2,1,...,-1.0,82.0,Q39,2,0,1,1,0,0,1


In [ ]:
#initial_df.to_json('final_train_df.json', orient="records", lines=True)
#files.download('final_train_df.json')

# Dev set

In [ ]:
dev_df = pd.read_csv('[MNLP 2025 HW1] train set [PUBLIC] - train_cleaned.tsv', sep='\t')

dev_df_subclass = pd.read_json('devset_subclass_instance.json', orient="records", lines=True)
dev_df_images = pd.read_json('dev_df_images.json', orient="records", lines=True)
dev_df_dates = pd.read_json('dev_df_dates.json', orient="records", lines=True)
dev_df_lang = pd.read_json('dev_df_lang.json', orient="records", lines=True)
dev_df_country = pd.read_json('dev_df_country.json', orient="records", lines=True)
dev_df_descr = pd.read_json('dev_df_descr_analyse.json', orient="records", lines=True)

## Merge all the datasets

In [ ]:
dev_df.columns.to_list()

['item', 'name', 'description', 'type', 'category', 'subcategory', 'label']

In [ ]:
# Get the dataset we first had
initial_dev_df = dev_df
initial_variables = dev_df.columns.to_list()

# Get the datasets with the features we created
df_to_merge = [dev_df_subclass, dev_df_images, dev_df_dates, dev_df_lang, dev_df_country, dev_df_descr]

# Merge all of them, in a single dataframe
for df in df_to_merge:
  initial_dev_df = pd.merge(left=initial_dev_df, right=df, on=initial_variables, how='inner')

# Visualisation :
initial_dev_df.head()

,item,name,description,type,category,subcategory,label,subclasses,instances_of,subclass_depth,image,date,nb_lang,main_country,descr_num_nouns,descr_num_verbs,descr_num_adjectives,descr_has_location,descr_has_ethnic_group,descr_has_event
0,http://www.wikidata.org/entity/Q306,Sebastián Piñera,Chilean entrepreneur and politician (1949–2024),entity,politics,politician,cultural exclusive,0,1,1,"[[[85, 93, 69], [64, 70, 47], [50, 55, 39], [1...",1949.0,126.0,Q298,2,0,1,0,1,0
1,http://www.wikidata.org/entity/Q12735,John Amos Comenius,"Czech teacher, educator, philosopher and write...",entity,politics,politician,cultural representative,0,1,1,"[[[33, 31, 17], [35, 33, 19], [32, 31, 19], [2...",1592.0,72.0,Q153136,4,0,1,0,1,0
2,http://www.wikidata.org/entity/Q1752,Macrinus,Roman emperor from 217 to 218,entity,politics,politician,cultural representative,0,1,1,"[[[109, 110, 103], [113, 115, 107], [117, 118,...",165.0,83.0,Q1747689,1,0,1,0,1,0
3,http://www.wikidata.org/entity/Q1639,Lamine Diack,Senegalese sports manager (1933–2021),entity,politics,politician,cultural representative,0,1,1,"[[[136, 91, 41], [123, 74, 26], [169, 134, 89]...",1933.0,42.0,Q1041,2,0,1,0,1,0
4,http://www.wikidata.org/entity/Q9588,Richard Nixon,President of the United States from 1969 to 1974,entity,politics,politician,cultural representative,0,1,1,"[[[106, 88, 76], [114, 96, 84], [115, 96, 84],...",1913.0,174.0,Q30,0,0,0,1,0,0


## Cleaning

### Type

We convert the variable 'type' to binary so that we can use it in our models.

In [ ]:
initial_dev_df['type_cat'] = initial_dev_df['type'].apply(lambda x: 1 if x == 'entity' else 0 if x == 'concept' else None)

In [ ]:
initial_dev_df['subcategory'].isna().sum()

### Category

We map the variable 'category' to multimodal variable, so that we can use it in our models.

In [ ]:
initial_dev_df["category_cat"] = initial_dev_df['category'].map(mapping_category)

NameError: name 'mapping_category' is not defined

In [ ]:
initial_dev_df['subcategory'].isna().sum()

### Subcategory

We map the variable 'category' to multimodal variable, so that we can use it in our models.

In [ ]:
initial_dev_df["subcategory_cat"] = initial_dev_df['subcategory'].map(mapping_subcategory)

politician : 0
political party : 1
government agency : 2
law : 3
policy : 4
government : 5
food : 6
drink : 7
dish : 8
ingredient : 9
cook : 10
cooking technique : 11
architectural structure : 12
architectural style : 13
architect : 14
construction : 15
building : 16
building material : 17
transport company : 18
automobile manufacturer : 19
transport : 20
mode of transport : 21
station : 22
athlete : 23
sports team : 24
sport : 25
sports club : 26
recurring sporting event : 27
sports equipment : 28
media company : 29
magazine : 30
television : 31
production company : 32
film studio : 33
streaming service : 34
philosopher : 35
religious leader : 36
religion : 37
philosophy : 38
philosophical movement : 39
religious movement : 40
poetry : 41
poet : 42
writing style : 43
literary genre : 44
literary award : 45
writer : 46
publisher : 47
artist : 48
visual arts : 49
art gallery : 50
art movement : 51
painting : 52
photographer : 53
clothing : 54
traditional costume : 55
designer : 56
fashi

In [ ]:
initial_dev_df['subcategory'].isna().sum()

### Images

Here, we have a lot of missing values, so we fill in the missing values with white images, in order to

- Hope that the model will learn to recognize and ignore thoses white images
- Interpret the fact that the absence of image for an item can be correlated to its label
- Regularization with noise

In [ ]:
def fill_NaN_image(image):

  """
  If a cell is empty, it fills in with a white image, of dimension (28,28,3) ie (height, width, channels)
  """

  if image is None or (isinstance(image, float) and np.isnan(image)):
    image = np.ones((28, 28, 3), dtype=np.uint8) * 255

  return image

In [ ]:
initial_dev_df['image'] = initial_dev_df['image'].apply(lambda x: fill_NaN_image(x))

In [ ]:
initial_dev_df

,item,name,description,type,category,subcategory,label,subclasses,instances_of,subclass_depth,image,date,nb_lang,main_country,descr_num_nouns,descr_num_verbs,descr_num_adjectives,descr_has_location,descr_has_ethnic_group,descr_has_event
0,http://www.wikidata.org/entity/Q306,Sebastián Piñera,Chilean entrepreneur and politician (1949–2024),entity,politics,politician,cultural exclusive,0,1,1,"[[[85, 93, 69], [64, 70, 47], [50, 55, 39], [1...",1949.0,126.0,Q298,2,0,1,0,1,0
1,http://www.wikidata.org/entity/Q12735,John Amos Comenius,"Czech teacher, educator, philosopher and write...",entity,politics,politician,cultural representative,0,1,1,"[[[33, 31, 17], [35, 33, 19], [32, 31, 19], [2...",1592.0,72.0,Q153136,4,0,1,0,1,0
2,http://www.wikidata.org/entity/Q1752,Macrinus,Roman emperor from 217 to 218,entity,politics,politician,cultural representative,0,1,1,"[[[109, 110, 103], [113, 115, 107], [117, 118,...",165.0,83.0,Q1747689,1,0,1,0,1,0
3,http://www.wikidata.org/entity/Q1639,Lamine Diack,Senegalese sports manager (1933–2021),entity,politics,politician,cultural representative,0,1,1,"[[[136, 91, 41], [123, 74, 26], [169, 134, 89]...",1933.0,42.0,Q1041,2,0,1,0,1,0
4,http://www.wikidata.org/entity/Q9588,Richard Nixon,President of the United States from 1969 to 1974,entity,politics,politician,cultural representative,0,1,1,"[[[106, 88, 76], [114, 96, 84], [115, 96, 84],...",1913.0,174.0,Q30,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6246,http://www.wikidata.org/entity/Q321103,Bühl,"quarter of Tübingen, Baden-Württemberg, Germany",entity,geography,neighborhood,cultural exclusive,0,2,1,"[[[255, 255, 255], [255, 255, 255], [255, 255,...",NaN,73.0,Q183,1,0,0,1,0,0
6247,http://www.wikidata.org/entity/Q338167,Tenderloin,area of New York City during the late 19th and...,entity,geography,neighborhood,cultural exclusive,0,1,1,"[[[255, 255, 255], [255, 255, 255], [255, 255,...",NaN,9.0,Q30,3,0,3,1,0,0
6248,http://www.wikidata.org/entity/Q66991,Schinznach-Dorf,former municipality and current district of Sc...,entity,geography,neighborhood,cultural exclusive,0,3,1,"[[[61, 117, 223], [64, 118, 226], [65, 119, 22...",NaN,30.0,Q39,2,0,2,1,0,0
6249,http://www.wikidata.org/entity/Q66922,Ependes,village and former municipality in Bois-d'Amon...,entity,geography,neighborhood,cultural exclusive,0,2,1,"[[[153, 147, 44], [209, 198, 54], [201, 192, 5...",NaN,82.0,Q39,2,0,1,1,0,0


### Dates

Max values

In [ ]:
current_year = int(2025)
initial_dev_df['date'] = initial_dev_df['date'].apply(lambda x: None if x > current_year else x)

Missing values

In [ ]:
initial_dev_df['date'] = initial_dev_df['date'].fillna(-1)

### Nb_lang

In [ ]:
med_value = initial_dev_df['nb_lang'].median()
initial_dev_df['nb_lang'] = initial_dev_df['nb_lang'].fillna(int(med_value))

### Main country

In [ ]:
initial_dev_df['main_country_cat'] = initial_dev_df['main_country'].apply(lambda x: 0 if pd.isna(x) else 1)

## Exportation

In [ ]:
# Visualisation

initial_dev_df

,item,name,description,type,category,subcategory,label,subclasses,instances_of,subclass_depth,...,date,nb_lang,main_country,descr_num_nouns,descr_num_verbs,descr_num_adjectives,descr_has_location,descr_has_ethnic_group,descr_has_event,main_country_cat
0,http://www.wikidata.org/entity/Q306,Sebastián Piñera,Chilean entrepreneur and politician (1949–2024),entity,politics,politician,cultural exclusive,0,1,1,...,1949.0,126.0,Q298,2,0,1,0,1,0,1
1,http://www.wikidata.org/entity/Q12735,John Amos Comenius,"Czech teacher, educator, philosopher and write...",entity,politics,politician,cultural representative,0,1,1,...,1592.0,72.0,Q153136,4,0,1,0,1,0,1
2,http://www.wikidata.org/entity/Q1752,Macrinus,Roman emperor from 217 to 218,entity,politics,politician,cultural representative,0,1,1,...,165.0,83.0,Q1747689,1,0,1,0,1,0,1
3,http://www.wikidata.org/entity/Q1639,Lamine Diack,Senegalese sports manager (1933–2021),entity,politics,politician,cultural representative,0,1,1,...,1933.0,42.0,Q1041,2,0,1,0,1,0,1
4,http://www.wikidata.org/entity/Q9588,Richard Nixon,President of the United States from 1969 to 1974,entity,politics,politician,cultural representative,0,1,1,...,1913.0,174.0,Q30,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6246,http://www.wikidata.org/entity/Q321103,Bühl,"quarter of Tübingen, Baden-Württemberg, Germany",entity,geography,neighborhood,cultural exclusive,0,2,1,...,-1.0,73.0,Q183,1,0,0,1,0,0,1
6247,http://www.wikidata.org/entity/Q338167,Tenderloin,area of New York City during the late 19th and...,entity,geography,neighborhood,cultural exclusive,0,1,1,...,-1.0,9.0,Q30,3,0,3,1,0,0,1
6248,http://www.wikidata.org/entity/Q66991,Schinznach-Dorf,former municipality and current district of Sc...,entity,geography,neighborhood,cultural exclusive,0,3,1,...,-1.0,30.0,Q39,2,0,2,1,0,0,1
6249,http://www.wikidata.org/entity/Q66922,Ependes,village and former municipality in Bois-d'Amon...,entity,geography,neighborhood,cultural exclusive,0,2,1,...,-1.0,82.0,Q39,2,0,1,1,0,0,1


In [ ]:
#initial_devdf.to_json('final_dev_df.json', orient="records", lines=True)
#files.download('final_dev_df.json')